# Form Parsing with Gemma 3 27B

**objective**: Incorporate form parsing to the workflow using Gemma 3 27B




In [1]:
import os, json, re
from llama_parse import LlamaParse
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.embeddings.google_genai import GoogleGenAIEmbedding
from llama_index.core import (
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage
)

from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Event,
    Context
)
from helper import get_gemini_api_key, get_llama_cloud_api_key
from IPython.display import display, HTML
from helper import extract_html_content
from llama_index.utils.workflow import draw_all_possible_flows

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
llama_cloud_api_key = get_llama_cloud_api_key()
gemini_api_key = get_gemini_api_key()

## Parsing an Application Form with LlamaParse

Let's create a parser with our new parsing instructions, including formatting instructions.

In [4]:
parser = LlamaParse(
    api_key=llama_cloud_api_key,
    base_url=os.getenv("LLAMA_CLOUD_BASE_URL"),
    result_type="markdown",
    system_prompt="This is a job application form. Create a list of all the fields that need to be filled in. Return a bulleted list of the fields ONLY."
)

In [5]:
result = parser.load_data("data/fake_application_form.pdf")[0]

Started parsing the file under job_id ece63326-668b-4364-ae04-e6c7d5283ec2


As print out the results, you can see it has pulled all the boxes in the form and correctly turned them into field names to be filled in.

In [6]:
print(result.text)

- First Name
- Last Name
- Email
- Phone
- Linkedin
- Project Portfolio
- Degree
- Graduation Date
- Current Job Title
- Current Employer
- Technical Skills
- Describe why you’re a good fit for this position
- Do you have 5 years of experience in React?


In [7]:
# Clear storage to force recreation with Google GenAI embeddings
import shutil
if os.path.exists("./storage"):
    shutil.rmtree("./storage")
    print("Cleared storage directory to use Google GenAI embeddings")


A useful thing LLMs can do is turn human-readable formats into machine-readable ones. You will ask the Gemma 3 27B model to turn the list into a JSON object with a list of fields.

In [8]:
llm = GoogleGenAI(model="models/gemma-3-27b-it", api_key=gemini_api_key)

In [9]:
raw_json = llm.complete(
    f"""
    This is a parsed form.
    Convert it into a JSON object containing only the list 
    of fields to be filled in, in the form {{ fields: [...] }}. 
    <form>{result.text}</form>. 
    Return JSON ONLY, no markdown."""
)

Here's the raw JSON:

In [10]:
raw_json.text

'```json\n{\n  "fields": [\n    "First Name",\n    "Last Name",\n    "Email",\n    "Phone",\n    "Linkedin",\n    "Project Portfolio",\n    "Degree",\n    "Graduation Date",\n    "Current Job Title",\n    "Current Employer",\n    "Technical Skills",\n    "Describe why you’re a good fit for this position",\n    "Do you have 5 years of experience in React?"\n  ]\n}\n```'

And you can print out the fields one by one.

In [11]:
# Extract JSON from markdown code blocks if present
def extract_json_from_text(text):
    # Try to find JSON in markdown code blocks
    json_match = re.search(r'```(?:json)?\s*(\{.*?\})\s*```', text, re.DOTALL)
    if json_match:
        return json_match.group(1)
    # If no code blocks, return the text as is
    return text

json_text = extract_json_from_text(raw_json.text)
fields = json.loads(json_text)["fields"]

for field in fields:
    print(field)

First Name
Last Name
Email
Phone
Linkedin
Project Portfolio
Degree
Graduation Date
Current Job Title
Current Employer
Technical Skills
Describe why you’re a good fit for this position
Do you have 5 years of experience in React?


Now that you know how to parse the job application form, you will add this processing to the workflow of lesson 3. You will do that in two steps as shown in this figure:

<img width="700" src="images/L4-diagrams.png">

## Adding a Form Parser to the Workflow (first update)

Now let's take the workflow you built in the last lesson and add your parser.

In [12]:
class ParseFormEvent(Event):
    application_form: str

class QueryEvent(Event):
    query: str

Your `set_up` step now emits a `ParseFormEvent` which triggers your new step, `parse_form`. For the moment you can leave the ask_questions step untouched, it will be updated in another section of this notebook.

In [13]:
class RAGWorkflow(Workflow):
    
    storage_dir = "./storage"
    llm: GoogleGenAI
    query_engine: VectorStoreIndex

    @step
    async def set_up(self, ctx: Context, ev: StartEvent) -> ParseFormEvent:

        if not ev.resume_file:
            raise ValueError("No resume file provided")

        if not ev.application_form:
            raise ValueError("No application form provided")

        # define the LLM to work with
        self.llm = GoogleGenAI(model="models/gemma-3-27b-it", api_key=gemini_api_key)

        # ingest the data and set up the query engine
        if os.path.exists(self.storage_dir):
            # you've already ingested the resume document
            storage_context = StorageContext.from_defaults(persist_dir=self.storage_dir)
            index = load_index_from_storage(storage_context)
        else:
            # parse and load the resume document
            documents = LlamaParse(
                api_key=llama_cloud_api_key,
                base_url=os.getenv("LLAMA_CLOUD_BASE_URL"),
                result_type="markdown",
                system_prompt="This is a resume, gather related facts together and format it as bullet points with headers"
            ).load_data(ev.resume_file)
            # embed and index the documents
            index = VectorStoreIndex.from_documents(
                documents,
                embed_model=GoogleGenAIEmbedding(model_name="models/text-embedding-004", api_key=gemini_api_key)
            )
            index.storage_context.persist(persist_dir=self.storage_dir)

        # create a query engine
        self.query_engine = index.as_query_engine(llm=self.llm, similarity_top_k=5)

        # you no longer need a query to be passed in, 
        # you'll be generating the queries instead 
        # let's pass the application form to a new step to parse it
        return ParseFormEvent(application_form=ev.application_form)

    @step
    async def parse_form(self, ctx: Context, ev: ParseFormEvent) -> QueryEvent:
        parser = LlamaParse(
            api_key=llama_cloud_api_key,
            base_url=os.getenv("LLAMA_CLOUD_BASE_URL"),
            result_type="markdown",
            system_prompt="This is a job application form. Create a list of all the fields that need to be filled in. Return a bulleted list of the fields ONLY."
        )

        # get the LLM to convert the parsed form into JSON
        result = parser.load_data(ev.application_form)[0]
        raw_json = self.llm.complete(
            f"""
            This is a parsed form. 
            Convert it into a JSON object containing only the list 
            of fields to be filled in, in the form {{ fields: [...] }}. 
            <form>{result.text}</form>. 
            Return JSON ONLY, no markdown.
            """)
        
        # Extract JSON from markdown code blocks if present
        def extract_json_from_text(text):
            json_match = re.search(r'```(?:json)?\s*(\{.*?\})\s*```', text, re.DOTALL)
            if json_match:
                return json_match.group(1)
            return text
        
        json_text = extract_json_from_text(raw_json.text)
        fields = json.loads(json_text)["fields"]

        for field in fields:
            print(field)
        return StopEvent(result="Dummy event")

    # will be edited in the next section
    @step
    async def ask_question(self, ctx: Context, ev: QueryEvent) -> StopEvent:
        response = self.query_engine.query(f"This is a question about the specific resume we have in our database: {ev.query}")
        return StopEvent(result=response.response)

In [14]:
w = RAGWorkflow(timeout=60, verbose=False)
result = await w.run(
    resume_file="data/fake_resume.pdf",
    application_form="data/fake_application_form.pdf"
)

Started parsing the file under job_id 4e968cc5-2056-4568-9131-521b2bb5ee87
Started parsing the file under job_id 8194f44f-79a6-4065-90fd-6b8360508b69
First Name
Last Name
Email
Phone
Linkedin
Project Portfolio
Degree
Graduation Date
Current Job Title
Current Employer
Technical Skills
Describe why you’re a good fit for this position
Do you have 5 years of experience in React?


## Generating Questions (second update)

Excellent! Your workflow knows what fields it needs answers for. In this next iteration, you can fire off one `QueryEvent` for each of the fields, so they'll be executed concurrently (we talked about doing concurrent steps in Lesson 2). 

<img width="700" src="images/L4-diag-2.png">


The changes you're going to make are:
* Generate a `QueryEvent` for each of the questions you pulled out of the form
* Create a `fill_in_application` step which will take all the responses to the questions and aggregate them into a coherent response
* Add a `ResponseEvent` to pass the results of queries to `fill_in_application`

In [15]:
class ParseFormEvent(Event):
    application_form: str

class QueryEvent(Event):
    query: str
    field: str

# new!
class ResponseEvent(Event):
    field: str
    response: str

In [16]:
class RAGWorkflow(Workflow):
    
    storage_dir = "./storage"
    llm: GoogleGenAI
    query_engine: VectorStoreIndex

    @step
    async def set_up(self, ctx: Context, ev: StartEvent) -> ParseFormEvent:

        if not ev.resume_file:
            raise ValueError("No resume file provided")

        if not ev.application_form:
            raise ValueError("No application form provided")

        # define the LLM to work with
        self.llm = GoogleGenAI(model="models/gemma-3-27b-it", api_key=gemini_api_key)

        # ingest the data and set up the query engine
        if os.path.exists(self.storage_dir):
            # you've already ingested the resume document
            storage_context = StorageContext.from_defaults(persist_dir=self.storage_dir)
            # Force use of Google GenAI embeddings when loading from storage
            index = load_index_from_storage(
                storage_context,
                embed_model=GoogleGenAIEmbedding(model_name="models/text-embedding-004", api_key=gemini_api_key)
            )
        else:
            # parse and load the resume document
            documents = LlamaParse(
                api_key=llama_cloud_api_key,
                base_url=os.getenv("LLAMA_CLOUD_BASE_URL"),
                result_type="markdown",
                system_prompt="This is a resume, gather related facts together and format it as bullet points with headers"
            ).load_data(ev.resume_file)
            # embed and index the documents
            index = VectorStoreIndex.from_documents(
                documents,
                embed_model=GoogleGenAIEmbedding(model_name="models/text-embedding-004", api_key=gemini_api_key)
            )
            index.storage_context.persist(persist_dir=self.storage_dir)

        # create a query engine
        self.query_engine = index.as_query_engine(llm=self.llm, similarity_top_k=5)

        # you no longer need a query to be passed in, 
        # you'll be generating the queries instead 
        # let's pass the application form to a new step to parse it
        return ParseFormEvent(application_form=ev.application_form)

    @step
    async def parse_form(self, ctx: Context, ev: ParseFormEvent) -> QueryEvent:
        parser = LlamaParse(
            api_key=llama_cloud_api_key,
            base_url=os.getenv("LLAMA_CLOUD_BASE_URL"),
            result_type="markdown",
            system_prompt="This is a job application form. Create a list of all the fields that need to be filled in. Return a bulleted list of the fields ONLY."
        )

        # get the LLM to convert the parsed form into JSON
        result = parser.load_data(ev.application_form)[0]
        raw_json = self.llm.complete(
            f"""
            This is a parsed form. 
            Convert it into a JSON object containing only the list 
            of fields to be filled in, in the form {{ fields: [...] }}. 
            <form>{result.text}</form>. 
            Return JSON ONLY, no markdown.
            """)
        
        # Extract JSON from markdown code blocks if present
        def extract_json_from_text(text):
            json_match = re.search(r'```(?:json)?\s*(\{.*?\})\s*```', text, re.DOTALL)
            if json_match:
                return json_match.group(1)
            return text
        
        json_text = extract_json_from_text(raw_json.text)
        fields = json.loads(json_text)["fields"]

        # new!
        # generate one query for each of the fields, and fire them off
        for field in fields:
            ctx.send_event(QueryEvent(
                field=field,
                query=f"How would you answer this question about the candidate? {field}"
            ))

        # store the number of fields so we know how many to wait for later
        await ctx.set("total_fields", len(fields))
        return

    @step
    async def ask_question(self, ctx: Context, ev: QueryEvent) -> ResponseEvent:
        response = self.query_engine.query(f"This is a question about the specific resume we have in our database: {ev.query}")
        return ResponseEvent(field=ev.field, response=response.response)

    # new!
    @step
    async def fill_in_application(self, ctx: Context, ev: ResponseEvent) -> StopEvent:
        # get the total number of fields to wait for
        total_fields = await ctx.get("total_fields")

        responses = ctx.collect_events(ev, [ResponseEvent] * total_fields)
        if responses is None:
            return None # do nothing if there's nothing to do yet

        # we've got all the responses!
        responseList = "\n".join("Field: " + r.field + "\n" + "Response: " + r.response for r in responses)

        result = self.llm.complete(f"""
            You are given a list of fields in an application form and responses to
            questions about those fields from a resume. Combine the two into a list of
            fields and succinct, factual answers to fill in those fields.

            <responses>
            {responseList}
            </responses>
        """)
        return StopEvent(result=result)

In [17]:
w = RAGWorkflow(timeout=120, verbose=False)
result = await w.run(
    resume_file="data/fake_resume.pdf",
    application_form="data/fake_application_form.pdf"
)
print(result)

Loading llama_index.core.storage.kvstore.simple_kvstore from ./storage\docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from ./storage\index_store.json.
Started parsing the file under job_id 61a4a9bc-ad83-4f10-9d57-27e62f5afc3b
Here's a list of fields and answers, combining the application form fields with the provided resume responses:

*   **First Name:** Sarah
*   **Last Name:** Chen
*   **Email:** sarah.chen@email.com
*   **Phone:** sarah.chen@email.com
*   **Linkedin:** linkedin.com/in/sarahchen
*   **Project Portfolio:** EcoTrack (React, Node.js, MongoDB - featured in TechCrunch), ChatFlow (React, WebSocket - 5000+ MAU)
*   **Degree:** Bachelor of Science in Computer Science (University of California, Berkeley)
*   **Graduation Date:** 2017
*   **Current Job Title:** Senior Full Stack Developer
*   **Current Employer:** TechFlow Solutions (San Francisco, CA)
*   **Technical Skills:** Vue.js, Django, React.js, Express.js, Docker, Kubernetes, AWS (EC2, S3, Lam

## Workflow Visualization

You can visualize the workflow you just created.

In [18]:
WORKFLOW_FILE = "workflows/form_parsing_workflow.html"
draw_all_possible_flows(w, filename=WORKFLOW_FILE)
html_content = extract_html_content(WORKFLOW_FILE)
display(HTML(html_content), metadata=dict(isolated=True))

workflows/form_parsing_workflow.html
